In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from sim_class import Simulation


class PIDController:
    def __init__(self, Kp, Ki, Kd, setpoint=0):
        """
        PID controller for a single axis.
        """
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.setpoint = setpoint
        self.integral = 0
        self.previous_error = 0

    def update(self, current_value):
        """
        Calculate the control signal based on the current value.
        """
        error = self.setpoint - current_value
        self.integral += error
        derivative = error - self.previous_error
        self.previous_error = error
        return self.Kp * error + self.Ki * self.integral + self.Kd * derivative


class OT2Env(gym.Env):
    def __init__(self, render=False, max_steps=10000):
        super(OT2Env, self).__init__()
        self.render_enabled = render  # Use render_enabled to track rendering
        self.max_steps = max_steps

        # Create the simulation environment
        self.sim = Simulation(num_agents=1, render=self.render_enabled)

        # Define action and observation spaces
        self.action_space = spaces.Box(low=np.array([-1, -1, -1, 0]), high=np.array([1, 1, 1, 1]), shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

        # PID Controllers for X, Y, Z axes
        self.pid_x = PIDController(Kp=1.0, Ki=0.1, Kd=0.01, setpoint=0.0)
        self.pid_y = PIDController(Kp=1.0, Ki=0.1, Kd=0.01, setpoint=0.0)
        self.pid_z = PIDController(Kp=1.0, Ki=0.1, Kd=0.01, setpoint=0.0)

        # Track steps and goal position
        self.steps = 0
        self.goal_position = None

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)

        # Set a random goal position within the valid range
        self.goal_position = np.array([
            np.random.uniform(-0.18700, 0.25300),
            np.random.uniform(-0.17050, 0.21950),
            np.random.uniform(0.16940, 0.28950)
        ], dtype=np.float32)

        # Update PID setpoints
        self.pid_x.setpoint = self.goal_position[0]
        self.pid_y.setpoint = self.goal_position[1]
        self.pid_z.setpoint = self.goal_position[2]

        # Reset the simulation environment
        observation = self.sim.reset(num_agents=1)

        # Extract pipette position and combine with goal position
        pipette_position = np.array(observation[f'robotId_{self.sim.robotIds[0]}']['pipette_position'], dtype=np.float32)
        observation = np.concatenate([pipette_position, self.goal_position], axis=0)

        self.steps = 0
        return observation, {}

    def step(self, action):
        # Extract current pipette position from the environment
        observation = self.sim.run([action])
        pipette_position = np.array(observation[f'robotId_{self.sim.robotIds[0]}']['pipette_position'], dtype=np.float32)

        # PID-controlled adjustment
        control_signal_x = self.pid_x.update(pipette_position[0])
        control_signal_y = self.pid_y.update(pipette_position[1])
        control_signal_z = self.pid_z.update(pipette_position[2])

        # Calculate new action based on PID output
        control_action = np.array([control_signal_x, control_signal_y, control_signal_z, 0.0], dtype=np.float32)

        # Execute the action in the simulation
        self.sim.run([control_action])

        # Update observation
        observation = np.concatenate([pipette_position, self.goal_position], axis=0)

        # Calculate reward based on distance to the goal
        reward = -np.linalg.norm(pipette_position - self.goal_position)

        # Check termination condition (10 mm accuracy requirement)
        distance = np.linalg.norm(pipette_position - self.goal_position)  # Distance in meters
        terminated = distance <= 0.001  # 10 mm accuracy

        if terminated:
            reward = 100  # Bonus for reaching the goal

        # Check truncation
        truncated = self.steps >= self.max_steps

        # Print debug information
        distance_mm = distance * 1000  # Convert distance to millimeters
        print(f"Step {self.steps}:")
        print(f"  Current Position: {pipette_position}")
        print(f"  Goal Position: {self.goal_position}")
        print(f"  Distance to Goal: {distance_mm:.2f} mm")

        self.steps += 1
        return observation, reward, terminated, truncated, {}

    def render(self):
        if self.render_enabled:
            self.sim.render()

    def close(self):
        self.sim.close()


# Testing Process
if __name__ == "__main__":
    env = OT2Env(render=True)
    observation, _ = env.reset()

    print(f"Goal Position: {env.goal_position}")

    for _ in range(10000):  # Simulate 50 steps
        action = env.action_space.sample()  # Example random action
        observation, reward, terminated, is_truncated, _ = env.step(action)

        if terminated or is_truncated:
            print("Goal Reached!" if terminated else "Simulation Truncated.")
            break

    env.close()


Goal Position: [-0.14755969  0.18229967  0.19934526]
Step 0:
  Current Position: [0.0735 0.0901 0.1195]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 252.47 mm
Step 1:
  Current Position: [0.0739 0.0904 0.1212]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 252.18 mm
Step 2:
  Current Position: [0.0735 0.0906 0.1222]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 251.45 mm
Step 3:
  Current Position: [0.0721 0.0921 0.1226]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 249.55 mm
Step 4:
  Current Position: [0.0698 0.0926 0.1245]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 246.77 mm
Step 5:
  Current Position: [0.0675 0.0931 0.125 ]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 244.41 mm
Step 6:
  Current Position: [0.0653 0.0936 0.1269]
  Goal Position: [-0.14755969  0.18229967  0.19934526]
  Distance to Goal: 241.7

: 